In [4]:
import numpy as np
from refnx._lib import flatten
from refnx.util import general

# Spatz distances
LS4 = 368.5
sample_distance = 6237
slit2_distance = 2327

In [13]:
def angler(angle, angles, slits=None):
    ss2hg = ss3hg = ss4hg = 0
    if slits is not None:
        ss2hg = slits[angle].get("ss2hg", 0)
        ss3hg = slits[angle].get("ss3hg", 0)
        ss4hg = slits[angle].get("ss4hg", 0)
    
    return f"omega_2theta {angles[angle]} {2 * angles[angle]} {ss2hg:.3f} {ss3hg:.3f} {ss4hg:.3f}"

def slit_optimiser(angles, footprint, resolution):
    slits = []
    for angle in angles:
        d1, d2 = general.slit_optimiser(
            footprint,
            resolution,
            angle,
            L12=L23,
            L2S=L3S,
            LS3=LS4,
            verbose=False,
        )
        d4 = general.height_of_beam_after_dx(d1, d2, L23, L3S + LS4)
        slits.append({"ss2hg": d1, "ss3hg": d2, "ss4hg": 1.2 * d4[1] + 2})
    return slits
        
def hslits(ss2hg, ss3hg, ss4hg):
    return f"hslits {ss2hg} {ss3hg} {ss4hg}"

def positioner(position, positions):
    offset_dct = positions[position]
    som_offset = offset_dct.get("som", 0)
    sxtop_offset = offset_dct.get("sxtop", 0)
    sx_offset = offset_dct.get("sx", 0)
    return [
        f"drive sxtop {sxtop_offset} sx {sx_offset}",
        f"rel som {som_offset}"
    ]

def acquire(time):
    return ["autosave 30", f"runscan dummy motor 0 0 1 time {time} force true"]

def syringe(pump0_rate, pump0_vol, pump1_rate, pump1_vol):
    cmds = [
        f"hset /sample/syr1/pump0/rat {pump0_rate}",
        f"hset /sample/syr1/pump0/vol {pump0_vol}",
        f"hset /sample/syr1/pump0/rat {pump1_rate}",
        f"hset /sample/syr1/pump0/vol {pump1_vol}",
        "hset /sample/syr1/pump0/run run",
    ]
    return cmds

def wait(time):
    return f"wait {time}"

def mvp(pos):
    if pos < 1 or pos > 6:
        raise ValueError("pos has to be an integer 1 <= pos <=6")
    return f"drive mvp_driveable {int(pos)}"

In [19]:
# alter ss3y
ss3y = 186.75
footprint = 50               # mm
resolution = 0.033           # mm


################################################
# don't alter
slit3_distance = sample_distance - (250 - ss3y)
L3S = sample_distance - slit3_distance
L23 = slit3_distance - slit2_distance
################################################

angles = [0.6, 3.2]
slits = slit_optimiser(angles, footprint, resolution)
positions = [
    {"sx": 0, "sc": 0, "som": 0.1, "sxtop": 0},  # 0
    {"sx": 0, "sc": 0, "som": 0, "sxtop": 0},    # 1
]

In [20]:
cmds = [
    mvp(1),
    syringe(2, 2, 0, 0),
    wait(120),
    angler(0, angles, slits),
    positioner(0, positions),
    acquire(3600),
    hslits(0, 0, 0)
]

In [21]:
print("\n".join(flatten(cmds)))

drive mvp_driveable 1
hset /sample/syr1/pump0/rat 2
hset /sample/syr1/pump0/vol 2
hset /sample/syr1/pump0/rat 0
hset /sample/syr1/pump0/vol 0
hset /sample/syr1/pump0/run run
wait 120
omega_2theta 0.6 1.2 1.894 0.484 2.902
drive sxtop 0 sx 0
rel som 0.1
autosave 30
runscan dummy motor 0 0 1 time 3600 force true
hslits 0 0 0
